# Introductory example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from nodepy import rk
import cvxpy as cp

import plot_fkt
plot_fkt.setup_plt()

import numpy.linalg as linalg

from numba import jit, float64

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

# Example I

In [ ]:
@jit(float64[:](float64,float64[:]),nopython=True)
def f_prod(t,u):
        du = np.zeros(4)
        du[0] = 0.01*u[1]+0.01*u[2]+0.003*u[3]-(u[0]*u[1])/(0.01+u[0])
        du[1] = (u[0]*u[1])/(0.01+u[0])-0.01*u[1]-0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.05*u[1]
        du[2] = 0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.01*u[2]-0.02*u[2]
        du[3] = 0.05*u[1] + 0.02*u[2]-0.003*u[3]
        return du
    
dt = 0.005 #standard
#dt = 0.00001

In [ ]:
#u0 = np.array([1.,1.,1.,1.])

u0 = np.array([8.,2.,1.,4.])



solver = Solver(rkm = dp5,
               dt = dt,
               t_final = 6,
               b_fixed=False,
               tol_neg=0,
               tol_change = 5,
               p = [4],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {'verbose_LP':False})

problem_reaction = Problem(f=f_prod,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

#t,u,b,KK = RK_variable_b(ssp104,dt,f_prod,u0,t_final=5,b_fixed=False,dumpK=True)
status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_reaction,verbose=False,dumpK=True)

t = np.array(t)
u_a = np.array(u).T
b_a = np.array(b).T
utils.show_status(status)

In [ ]:
u0 = np.array([8.,2.,1.,4.])
 #ssp104

solver = Solver(rkm = dp5,
               dt = dt,
               t_final = 6,
               b_fixed=True,
               tol_neg=0,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {'verbose_LP':False})

problem_reaction = Problem(f=f_prod,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

#t,u,b,KK = RK_variable_b(ssp104,dt,f_prod,u0,t_final=5,b_fixed=False,dumpK=True)
status_r,t,u,b,KK = RK_integrate(solver=solver,problem=problem_reaction,verbose=False,dumpK=True)

t = np.array(t)
u_r = np.array(u).T
b_r = np.array(b).T
utils.show_status(status_r)

In [ ]:
fig = plt.figure(figsize=[6.4*1.5, 4*1.5])

gs1 = gridspec.GridSpec(3, 2)
ax1 = fig.add_subplot(gs1[0:2,0])
ax2 = fig.add_subplot(gs1[2,0])

#fig, (ax1,ax2) = plt.subplots(2,1)

ax1.plot(t,u_a[0,:],'-C1',label='$u_1$')
ax1.plot(t,u_a[1,:],'-C2',label='$u_2$')
ax1.plot(t,u_a[2,:],'-C3',label='$u_3$')
ax1.plot(t,u_a[3,:],'-C4',label='$u_4$')


ax1.plot(t,u_r[0,:],'--C1',label='$u_1$')
ax1.plot(t,u_r[1,:],'--C2',label='$u_2$')
ax1.plot(t,u_r[2,:],'--C3',label='$u_3$')
ax1.plot(t,u_r[3,:],'--C4',label='$u_4$')


ax1.set_ylim(-2,11)

ax2.plot(t,np.linalg.norm((b_r-b_a),axis=0,ord=1),'-C5')
#ax2.set_ylim(-0.4,0.8)



ax1.grid()
ax2.grid()


ax2.set_xlabel('$t$')
ax1.set_ylabel('$u$')
ax2.set_ylabel(r'$\| \tilde{b} - b \|_1 $')


gs1.tight_layout(fig, rect=[0, 0, 1.6, 1])

fig.legend(loc=1)

plt.savefig('exampleI.pdf')

In [ ]:
plt.plot(b_a.T)

In [ ]:
#get time of first negative value
print(np.min(t[np.min(u_r,axis=0)<0]))
#get maximum value for cahnged b
print(np.max(t[b_r[1,:]!=b_a[1,:]]))

In [ ]:
t.shape

In [ ]:
print(utils.get_max_iter_h(status))

In [ ]:
'c' in status['b']

In [ ]:
dt = np.array([0,1,2])
np.zeros_like(dt,dtype=bool)

In [ ]:
np.save('b_ex1.npy', b_a[:,np.nonzero(dp5.b[0]!=b_a[0,:])])

In [ ]:
b_a[:,381]

In [ ]:
b_a[:,np.nonzero(dp5.b[0]!=b_a[0,:])]